In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
import threading
import time
import concurrent
import os
import csv
import pyarrow.feather as feather

from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import STCIndicator

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
outdir = ['OHLC5m','OHLC30m','OHLC1h','OHLC1D','STC','EMA','CPR','IB']
for folder in outdir:
    if not os.path.exists(folder):
        os.mkdir(folder)


In [ ]:
tickers = []
with open('symbols.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        tickers.append(row[0])
    tickers.pop(0) #remove zero index

In [ ]:
def fetch_ohlc(symbol, timeframe, timesymbol):

        symbol = symbol
        timeframe = timeframe
        timesymbol = timesymbol

        # fetch data-binance api
        candlestick_url = (
            "https://fapi.binance.com/fapi/v1/continuousKlines?pair="
            + symbol
            + "&contractType=PERPETUAL&interval="
            + str(timeframe)
            + timesymbol
            + "&limit=1500"
        )
        candlestick_chart = requests.get(candlestick_url).json()
        candlestick_df = pd.DataFrame(candlestick_chart)
        candlestick_df = candlestick_df.iloc[:, 1:7]
        candlestick_df.columns = ["open", "high", "low", "close", "volume", "date"]
        candlestick_df["date"] = pd.to_datetime(
            candlestick_df["date"], unit="ms"
        ).round("1s")
        candlestick_df.insert(0, "date", candlestick_df.pop("date"))

        # reset to midnight
        candlestick_df.date = pd.to_datetime(candlestick_df.date)
        min_date = candlestick_df.date.min()
        NextDay_Date = (min_date + datetime.timedelta(days=1)).replace(
            hour=0, minute=0, second=0, microsecond=0
        )
        candlestick_df = candlestick_df[candlestick_df.date >= NextDay_Date].copy()

        candlestick_df = candlestick_df.set_index("date")
        candlestick_df["symbol"] = symbol

        ohlc_data = candlestick_df

        cols = ["open", "high", "low", "close", "volume"]
        ohlc_data[cols] = ohlc_data[cols].apply(pd.to_numeric, errors="coerce")
        ohlc_data[cols] = ohlc_data[cols].round(decimals=8) #decimal important for small alts
        return ohlc_data

In [ ]:
def ohlc_smaller(symbol):
    smaller = fetch_ohlc(symbol,5,'m')
    smaller.to_csv(f"OHLC5m/"+symbol+"5m.csv")

def ohlc_larger(symbol):
    try:
        larger = fetch_ohlc(symbol,1,'h')
        larger.to_csv(f"OHLC1h/"+symbol+"1h.csv")
    except ValueError:
        pass

#optional daily ohlc data
def ohlc_daily(symbol):
    try:
        daily = fetch_ohlc(symbol,1,'d')
        daily.to_csv(f"OHLC1D/"+symbol+"1d.csv")
    except ValueError:
        pass

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:
     result = [executor.submit(ohlc_smaller,symbol) 
               for symbol in tickers]

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:
     result = [executor.submit(ohlc_larger,symbol) 
               for symbol in tickers]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=616d9311-e28a-4f68-a1e2-cfa37b447691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>